In [1]:
pip install evaluate rouge-score transformers nltk peft

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 12.3 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=764e19e14645bfd7a7ea9bba880cf880bf9c8491d697ce7550693cc915686186
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import LoraConfig, get_peft_model, TaskType, PeftConfig, PeftModel
from nltk.tokenize import sent_tokenize
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import evaluate
import torch


dataset_name = "bitext/Bitext-customer-support-llm-chatbot-training-dataset"
dataset = load_dataset(dataset_name, split='train')
df_train = dataset.to_pandas()
train_data, eval_data = train_test_split(df_train, test_size=0.2, random_state=42)

# Set up model and tokenizer
model_id = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Preprocess function
def preprocess_function(sample, padding="max_length"):
    model_inputs = tokenizer(sample["instruction"], max_length=256, padding=padding, truncation=True)
    labels = tokenizer(sample["response"], max_length=256, padding=padding, truncation=True)
    if padding == "max_length":
        labels["input_ids"] = [[(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize datasets
train_tokenized_dataset = Dataset.from_pandas(train_data).map(preprocess_function, batched=True, remove_columns=['flags', 'instruction', 'category', 'intent', 'response'])
test_tokenized_dataset = Dataset.from_pandas(eval_data).map(preprocess_function, batched=True, remove_columns=['flags', 'instruction', 'category', 'intent', 'response'])

# Set up LoRA configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Set up data collator and training arguments
label_pad_token_id = -100
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
)
output_dir = "/kaggle/working/lora-flan-t5-small-chat"
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-3,
    num_train_epochs=3,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="epoch",
    save_strategy="epoch",
    push_to_hub=False
)

# Set up trainer and train the model
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_tokenized_dataset,
    eval_dataset=test_tokenized_dataset
)
model.config.use_cache = False
trainer.train()

README.md:   0%|          | 0.00/11.9k [00:00<?, ?B/s]

(…)t_Training_Dataset_27K_responses-v11.csv:   0%|          | 0.00/19.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26872 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
1344,1.792400
2688,1.515400
4032,1.445300


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=4032, training_loss=1.5843820117768788, metrics={'train_runtime': 1714.3001, 'train_samples_per_second': 37.619, 'train_steps_per_second': 2.352, 'total_flos': 6062296973967360.0, 'train_loss': 1.5843820117768788, 'epoch': 3.0})

In [ ]:
# # Save the model using the recommended method
# output_dir = "/kaggle/working/flan_t5/lora-flan-t5-small-chat"
# os.makedirs(output_dir, exist_ok=True)
# trainer.model.save_pretrained(output_dir)
# trainer.tokenizer.save_pretrained(output_dir)

In [4]:
# Load the original model and LoRA fine-tuned model
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_id).cuda()
original_tokenizer = AutoTokenizer.from_pretrained(model_id)

peft_model_id = "/kaggle/working/lora-flan-t5-small-chat/checkpoint-4032/"
config = PeftConfig.from_pretrained(peft_model_id)
peft_model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
peft_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
peft_model = PeftModel.from_pretrained(peft_model, peft_model_id, device_map={"":0}).cuda()
peft_model.eval()

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 512)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 512)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=512, out_features=384, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=16, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=16, out_features=384, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
             

In [5]:
# Prepare test data for ROUGE evaluation
test_data = eval_data['instruction'].tolist()[:150]
response = eval_data['response'].tolist()[:150]

# Generate responses using both models
def generate_responses(model, tokenizer, inputs):
    responses = []
    for inp in inputs:
        input_ids = tokenizer(inp, return_tensors="pt", truncation=True, max_length=256).input_ids.cuda()
        outputs = model.generate(input_ids=input_ids, do_sample=True, top_p=0.9, max_length=256)
        responses.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
    return responses

original_model_responses = generate_responses(original_model, original_tokenizer, test_data)
peft_model_responses = generate_responses(peft_model, peft_tokenizer, test_data)

# Compute ROUGE scores
rouge = evaluate.load('rouge')
original_model_results = rouge.compute(
    predictions=original_model_responses,
    references=response,
    use_aggregator=True,
    use_stemmer=True,
)
peft_model_results = rouge.compute(
    predictions=peft_model_responses,
    references=response,
    use_aggregator=True,
    use_stemmer=True,
)

# Print ROUGE scores
print('\nOriginal Model ROUGE Scores:')
print(original_model_results)
print('\nPEFT Model ROUGE Scores:')
print(peft_model_results)



Original Model ROUGE Scores:
{'rouge1': 0.07738113835765134, 'rouge2': 0.017617387788871293, 'rougeL': 0.06259720008228106, 'rougeLsum': 0.06401729186231758}

PEFT Model ROUGE Scores:
{'rouge1': 0.44270525310677267, 'rouge2': 0.181763302085444, 'rougeL': 0.2878179321772827, 'rougeLsum': 0.30936962797949247}


In [7]:
# Compute BLEU scores
bleu = evaluate.load('bleu')
original_model_bleu = bleu.compute(
    predictions=original_model_responses,  # Keep the original model responses as strings
    references=[[ref] for ref in response]  # Each reference should be a list of strings
)
peft_model_bleu = bleu.compute(
    predictions=peft_model_responses,
    references=[[ref] for ref in response]  # Each reference should be a list of strings
)

# Print BLEU scores
print('\nOriginal Model BLEU Score:')
print(original_model_bleu)

print('\nPEFT Model BLEU Score:')
print(peft_model_bleu)


Original Model BLEU Score:
{'bleu': 2.6740041175707e-07, 'precisions': [0.4480836236933798, 0.08786936236391912, 0.029488291413703384, 0.015518913676042677], 'brevity_penalty': 4.104263196064477e-06, 'length_ratio': 0.07460746594572112, 'translation_length': 1435, 'reference_length': 19234}

PEFT Model BLEU Score:
{'bleu': 0.13296333953604048, 'precisions': [0.4446771587577045, 0.18507336513495562, 0.10499055511547133, 0.06617059221450096], 'brevity_penalty': 0.8598662737517796, 'length_ratio': 0.868826037225746, 'translation_length': 16711, 'reference_length': 19234}
